In [1]:
from aiModules import *
from databaseModules import *

Database connection established.
Successfully retrieved metadata and inspector.


In [4]:
reepicheep = lb.ticketAssistant_lc()

In [5]:
blind_run = reepicheep.run_blind()

[{'role': 'user', 'content': 'I need to write a user story for my client WK'}, <OpenAIObject at 0x20eb1850dd0> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "getUserStoryTemplate",
    "arguments": "{\n  \"client\": \"WK\"\n}"
  }
}, {'role': 'function', 'name': 'getUserStoryTemplate', 'content': [SystemMessage(content='You are an AI Jira ticket writing assistant that specializes in ticket generation and refinement. You are currently assisting a WK stakeholder by constructing acceptance criteria for a user story ticket.', additional_kwargs={}), HumanMessage(content='I am a business analyst stakeholder for WK. I need to write the acceptance criteria ticket for a user story.', additional_kwargs={}, example=False)]}]
[SystemMessage(content='You are an AI Jira ticket writing assistant that specializes in ticket generation and refinement. You are currently assisting a WK stakeholder by constructing acceptance criteria for a user story ticket.', additiona

In [8]:
initial_result = reepicheep.ticketOutputs[0]

print(initial_result.content)

{
    1: "The subscription table is successfully created in the database",
    2: "The subscription table is structured according to the tech design schema",
    3: "The subscription table contains an ID column, a foreign key to both the users and products tables, and columns for start and end dates",
    4: "Any data entered into the subscription table is valid and successfully stored in the database"
}


In [9]:
markdown_result = reepicheep.run_context()

Human: Please write out the AC in Markdown so that I can paste it into a Jira panel
AI:   Sure thing! Here is the acceptance criteria in Markdown format: 

**Acceptance Criteria** 
1. The subscription table is successfully created in the database
2. The subscription table is structured according to the tech design schema
3. The subscription table contains an ID column, a foreign key to both the users and products tables, and columns for start and end dates
4. Any data entered into the subscription table is valid and successfully stored in the database.

Is there anything else I can help you with?
Exiting...
{'ticket_type': 'feature', 'result_type': 'acceptance_criteria', 'format_type': 'markdown'}
[{'role': 'user', 'content': ''}, {'role': 'assistant', 'content': ' Hi! How can I help you?'}, {'role': 'user', 'content': 'I need to add a table called subscriptions to my database'}, {'role': 'assistant', 'content': " Sure thing! Adding a subscription table to your database should be relat

In [16]:
markdown_output = reepicheep.ticketOutputs[1]

print(reepicheep.ticketOutputs[1].content)

**Acceptance Criteria**

1. The subscription table is successfully created in the database
2. The subscription table is structured according to the tech design schema
3. The subscription table contains an ID column, a foreign key to both the users and products tables, and columns for start and end dates
4. Any data entered into the subscription table is valid and successfully stored in the database.


In [17]:
sql_result = reepicheep.run_context()

Human: Give me the create script and a select script for the new table with fk references
AI:  Sure thing! Here is the CREATE script for the subscription table and a SELECT script for the table with foreign key references: 

CREATE TABLE subscriptions 
(
  id INTEGER PRIMARY KEY, 
  user_id INTEGER NOT NULL REFERENCES users(id),
  product_id INTEGER NOT NULL REFERENCES products(id),
  start_date DATETIME NOT NULL,
  end_date DATETIME NOT NULL
);

SELECT * 
FROM subscriptions s 
INNER JOIN users u 
ON s.user_id = u.id 
INNER JOIN products p 
ON s.product_id = p.id;

Is there anything else I can help you with?
Exiting...
{'ticket_type': 'feature', 'result_type': 'output', 'format_type': 'markdown'}
[{'role': 'user', 'content': ''}, {'role': 'assistant', 'content': ' Hi! How can I help you?'}, {'role': 'user', 'content': 'I need to add a table called subscriptions to my database'}, {'role': 'assistant', 'content': " Sure thing! Adding a subscription table to your database should be relati

In [18]:
sql_output = reepicheep.ticketOutputs[2]
print(sql_output.content)

## Feature Ticket: Add Subscription Table to Database

### Description
As a developer, I need to add a table called "subscriptions" to the database. This table will store information about user subscriptions including an ID, foreign keys to the "users" and "products" tables, and start and end dates for each subscription.

### Acceptance Criteria
1. The subscription table is successfully created in the database.
2. The subscription table is structured according to the tech design schema.
3. The subscription table contains an ID column, a foreign key to both the users and products tables, and columns for start and end dates.
4. Any data entered into the subscription table is valid and successfully stored in the database.

### Technical Details
```sql
CREATE TABLE subscriptions 
(
  id INTEGER PRIMARY KEY, 
  user_id INTEGER NOT NULL REFERENCES users(id),
  product_id INTEGER NOT NULL REFERENCES products(id),
  start_date DATETIME NOT NULL,
  end_date DATETIME NOT NULL
);

SELECT * 
FROM 

In [44]:
# need to figure out the best way to prompt for a clean SQL output
import sqlalchemy as sa

create_script = sa.text(
    """CREATE TABLE subscriptions 
    (
      id INTEGER PRIMARY KEY, 
      user_id INTEGER NOT NULL REFERENCES users(id),
      product_id INTEGER NOT NULL REFERENCES products(id),
      start_date DATETIME NOT NULL,
      end_date DATETIME NOT NULL
    );"""
)
    
select_script = sa.text(
    """SELECT * 
    FROM userorderProducts u
    INNER JOIN products p 
    ON u.product_id = p.id;"""
)

scripts = [create_script, select_script]

print(scripts)

[<sqlalchemy.sql.elements.TextClause object at 0x0000020EB33FBB10>, <sqlalchemy.sql.elements.TextClause object at 0x0000020EB33FAFD0>]


In [60]:
validated_queries = dbvf.validateTables(scripts[1].text)

['userorderproducts', 'products'] 

Invalid query. The table 'userorderproducts' does not exist in the database.
Replaced userorderproducts with userorderitems 

products is valid.

['userorderitems', 'products', 'There is no specific information provided in the given schema to predict the correct table name for subscriptions.', 'users']


In [61]:
print(validated_queries)

SELECT * 
    FROM userorderitems u
    INNER JOIN products p 
    ON u.product_id = p.id;


In [65]:
validated_columns = dbvf.validateColumns(validated_queries.text)

Query columns by table: 
 {'userorderitems': ['product_id'], 'products': ['id']} 

id is valid.

Invalid query. The column 'product_id' does not exist in the table 'UserOrderItems'.
Replaced 'product_id' with 'productid' 

SELECT * 
    FROM userorderitems u
    INNER JOIN products p 
    ON u.productid = p.id;
